In [30]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
import io
import pandas as pd
from collections import Counter
import requests
from bs4 import BeautifulSoup as bs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
import tika
import pickle
from tika import parser
import spacy
from spacy import displacy
nlp = spacy.load("en_core_web_sm")
nlp.max_length = 2000000
import re
import pickle

import matplotlib.pyplot as plt
%matplotlib notebook
%matplotlib inline

## Part 1: Get folder links for four submitter categories

In [31]:
#get download links for all pdfs in RH-002-2020 proceeding

REGDOCS_url_main = 'https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadResult/3890507' 
cookies = {'RDI-NumberOfRecords' : '200'} 

r = requests.get(REGDOCS_url_main, cookies = cookies)
soup = bs(r.text, 'html.parser')

a_html_tags = [x['href'] for x in soup.find_all('a', {'class' : 'link-wait no-expand'})]
a_html_tags = ['https://apps.cer-rec.gc.ca' + x for x in a_html_tags]

submitter_category = [x.text for x in soup.find_all('a', {'class' : 'link-wait no-expand'})]


df_url_main = pd.DataFrame({'submitter_category' : submitter_category, 'folder_link' : a_html_tags})
df_url_main['folder_link'] = df_url_main['folder_link'].str.replace('View', 'LoadResult')
df_url_main = df_url_main.append({'submitter_category' : 'Transcripts', 'folder_link' : 'https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadResult/4096405'}, ignore_index=True)
df_url_main.head()
df_url_main.shape

,submitter_category,folder_link
0,A - Commission,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...
1,B - Enbridge Pipeline Inc.,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...
2,C - Intervenors,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...
3,D - Commenters,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...
4,Transcripts,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...


(5, 2)

## Part 2: Get download links and document attributes for each submitter category

In [32]:
%%time
# CPU times: user 25.7 s, sys: 974 ms, total: 26.7 s
# Wall time: 6min 27s

docs_df = []

for a, b in zip(df_url_main['submitter_category'], df_url_main['folder_link']):
    if a == 'A - Commission' or a == 'B - Enbridge Pipeline Inc.':
        r = requests.get(b, cookies = cookies)
        soup = bs(r.text, 'html.parser')
        a_tags = [x['href'] for x in soup.find_all('a', {'class' : 'link-wait no-expand'})]
        a_tags_full = ['https://apps.cer-rec.gc.ca' + x for x in a_tags]
        
        folder_link = [x.replace('/View', '/LoadResult') for x in a_tags_full]
        folder_name = [x.text for x in soup.find_all('a', {'class' : 'link-wait no-expand'})]
        submitter = [x.text for x in soup.find_all('td', {'class' : 'col-md-2'})]
        
        date = [x.text for x in soup.find_all('td', {'class' : 'nowrap'})]
        
#         for x, y, z, m, n in zip(folder_link[0:2], submitter[0:2], date[0:2], folder_name[0:2], folder_link[0:2] ):
        for x, y, z, m, n in zip(folder_link, submitter, date, folder_name, folder_link):
            r = requests.get(x, cookies = cookies)
            soup = bs(r.text, 'html.parser')
            a_tags = [x['href'] for x in soup.find_all('a', {'target' : '_blank'})]
            download_link = ['https://apps.cer-rec.gc.ca' + x for x in a_tags]
            
            doc_name = [x.text for x in soup.find_all('a', {'target' : '_blank'})]

            df = pd.DataFrame({'doc_name' : doc_name, 'download_link' : download_link, 'submitter_name' : y, \
                                'filing_date' : z, 'folder_name' : m, 'submitter_category': a, 'folder_link' : n })
            docs_df.append(df)

    if a == 'C - Intervenors':
        r = requests.get(b, cookies = cookies)
        soup = bs(r.text, 'html.parser')
        intervener_name = [x.text for x in soup.find_all('a' ,{'class' : 'link-wait no-expand' })]
        interveners_folders = [x['href'] for x in soup.find_all('a' ,{'class' : 'link-wait no-expand' })]
        interveners_folders = [x.replace('View', 'LoadResult') for x in interveners_folders]
        interveners_folders = ['https://apps.cer-rec.gc.ca' + x for x in interveners_folders]
        date = [x.text for x in soup.find_all('td', {'class' : 'nowrap'})]

#         for x, y, z in zip(interveners_folders[0:2], intervener_name[0:2], date[0:2]):
        for x, y, z in zip(interveners_folders, intervener_name, date):

            r = requests.get(x)
            soup = bs(r.text, 'html.parser')
            folder_name = [x.text for x in soup.find_all('a' ,{'class' : 'link-wait no-expand' })]
            folder_link = [x['href'] for x in soup.find_all('a' ,{'class' : 'link-wait no-expand' })]
            folder_link = [x.replace('View', 'LoadResult') for x in folder_link]
            folder_link = ['https://apps.cer-rec.gc.ca' + x for x in folder_link]
            
#             for g, h in zip(folder_link[0:2], folder_name[0:2]):
            for g, h in zip(folder_link, folder_name):
                r = requests.get(g, cookies = cookies)
                soup = bs(r.text, 'html.parser')
                a_tags = [x['href'] for x in soup.find_all('a', {'target' : '_blank'})]
                download_link = ['https://apps.cer-rec.gc.ca' + x for x in a_tags] 
                
                doc_name = [x.text for x in soup.find_all('a', {'target' : '_blank'})]
                
                df = pd.DataFrame({'doc_name' : doc_name, 'download_link' : download_link, \
                                    'filing_date' : z, 'folder_name' : h, 'submitter_category': a, \
                                   'submitter_name' : y, 'folder_link' : g })
           
                docs_df.append(df)
    if a == 'Transcripts':
        r = requests.get(b, cookies = cookies)
        soup = bs(r.text, 'html.parser')
        folder_name = [x.text for x in soup.find_all('a' ,{'class' : 'link-wait no-expand' })]
#         print(folder_name)
        folder_link = [x['href'] for x in soup.find_all('a' ,{'class' : 'link-wait no-expand' })]
        folder_link = [x.replace('View', 'LoadResult') for x in folder_link]
        folder_link = ['https://apps.cer-rec.gc.ca' + x for x in folder_link]
        date = [x.text for x in soup.find_all('td', {'class' : 'nowrap'})]
        submitter = [x.text for x in soup.find_all('td', {'class' : 'col-md-2'})]
        
        
        for g, h, i, j in zip(folder_link, folder_name, date, submitter):
            r = requests.get(g, cookies = cookies)
            soup = bs(r.text, 'html.parser')
            a_tags = [x['href'] for x in soup.find_all('a', {'target' : '_blank'})]
#             print(a_tags)
            download_link = ['https://apps.cer-rec.gc.ca' + x for x in a_tags] 
            doc_name = [x.text for x in soup.find_all('a', {'target' : '_blank'})]
            df = pd.DataFrame({'doc_name' : doc_name, 'download_link' : download_link, \
                                    'filing_date' : i, 'folder_name' : h, 'submitter_category': a, \
                                   'submitter_name' : j, 'folder_link' : g })
            docs_df.append(df)

Wall time: 1min 44s


In [33]:
df_attributes = pd.concat(docs_df, ignore_index = True)
df_attributes.to_csv('flatdata.csv', encoding = 'utf-8-sig')

ipykernel_launcher:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [34]:
# create and save a dataframe of attributes
df_attributes = pd.concat(docs_df, ignore_index = True)
df_attributes = df_attributes[~df_attributes['doc_name'].str.contains('Receipt|Corrections')]

df_attributes['submitter_name'] = df_attributes['submitter_name'].fillna(df_attributes['submitter_category'])
df_attributes.to_csv('RH0012020_attributes.csv', index = 'False')
df_attributes['download_link'].nunique()
df_attributes.shape
df_attributes.head(5)
df_attributes.tail(5)

ipykernel_launcher:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



1218

(1222, 7)

,doc_name,download_link,filing_date,folder_link,folder_name,submitter_category,submitter_name
0,C13853-1 Commission – Procedural Update No. 6 ...,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,2021-06-30,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,C13853 Commission | Commission - Procedural Up...,A - Commission,Commission | Commission
2,C13649-1 Commission – Letter - Enbridge – Cana...,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,2021-06-19,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,C13649 Commission | Commission - Letter – Enbr...,A - Commission,Commission | Commission
3,C13649-2 Commission – Aid to Questioning – Enb...,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,2021-06-19,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,C13649 Commission | Commission - Letter – Enbr...,A - Commission,Commission | Commission
5,C13454-1 Commission – Procedural Update No. 5 ...,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,2021-06-08,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,C13454 Commission | Commission - Procedural Up...,A - Commission,Commission | Commission
7,C13445-1 Commission – Letter – Enbridge – Cana...,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,2021-06-07,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,C13445 Commission | Commission - Letter – Enbr...,A - Commission,Commission | Commission


,doc_name,download_link,filing_date,folder_link,folder_name,submitter_category,submitter_name
1907,"C14074-1 Vol.25-TueJuly13,2021 - A7V2E9",https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,2021-07-13,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,C14074 International Reporting Inc. - RH-001-2...,Transcripts,International Reporting Inc.
1909,"C14088-1 Vol.26-WedJuly14,2021 - A7V2I1",https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,2021-07-14,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,C14088 International Reporting Inc. - RH-001-2...,Transcripts,International Reporting Inc.
1914,"C14110-1 Vol.27-ThuJuly15,2021 - A7V2R7",https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,2021-07-15,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,C14110 International Reporting Inc. - RH-001-2...,Transcripts,International Reporting Inc.
1919,C14113-1 EPAC Letter to CER regarding correct...,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,2021-07-16,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,C14113 The Explorers and Producers Association...,Transcripts,The Explorers and Producers Association of Canada
1927,"C14134-1 Vol.28-FriJuly16,2021 - A7V2Y3",https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,2021-07-16,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,C14134 International Reporting Inc. - RH-001-2...,Transcripts,International Reporting Inc.


## Part 3: Download pdfs, convert to string, save as pickle file

In [35]:
%%time 
# CPU times: user 2min 15s, sys: 18.2 s, total: 2min 33s
# Wall time: 1h 20min 9s; 24 May 2021
list_strings = []
bad_files = []

for x in df_attributes['download_link'][:10]:
    try:
        pdf_dict = parser.from_file(x) #start of Tika; type string
        raw_string = pdf_dict['content']
#         print(raw_string)
        df_ = pd.DataFrame([raw_string], columns = ['raw_string'])
        df_['download_link'] = x
        
        list_strings.append(df_)                    
    except: 
        bad_files.append(x)
        
with open('C:/Users/t1nipun/Desktop/files/RH0012020_strings.txt', 'wb') as fp:  ## concatenate text strings in a .txt file from all the PDFs
    pickle.dump(list_strings, fp)

2021-08-12 19:04:27,972 [MainThread  ] [INFO ]  Retrieving https://apps.cer-rec.gc.ca/REGDOCS/File/Download/4103988 to C:\Users\t1nipun\AppData\Local\Temp\11/regdocs-file-download-4103988.
2021-08-12 19:04:29,633 [MainThread  ] [INFO ]  Retrieving https://apps.cer-rec.gc.ca/REGDOCS/File/Download/4102482 to C:\Users\t1nipun\AppData\Local\Temp\11/regdocs-file-download-4102482.
2021-08-12 19:04:31,237 [MainThread  ] [INFO ]  Retrieving https://apps.cer-rec.gc.ca/REGDOCS/File/Download/4102483 to C:\Users\t1nipun\AppData\Local\Temp\11/regdocs-file-download-4102483.
2021-08-12 19:04:32,873 [MainThread  ] [INFO ]  Retrieving https://apps.cer-rec.gc.ca/REGDOCS/File/Download/4101249 to C:\Users\t1nipun\AppData\Local\Temp\11/regdocs-file-download-4101249.
2021-08-12 19:04:34,502 [MainThread  ] [INFO ]  Retrieving https://apps.cer-rec.gc.ca/REGDOCS/File/Download/4099449 to C:\Users\t1nipun\AppData\Local\Temp\11/regdocs-file-download-4099449.
2021-08-12 19:04:36,094 [MainThread  ] [INFO ]  Retriev

## Part 4: Spacy - sentence tokens 

In [28]:
#load data and clean up
with open ('C:/Users/t1nipun/Desktop/files/RH0012020_strings.txt', 'rb') as file:
    pdf_strings = pickle.load(file)
    
len(pdf_strings)
pdf_strings_df = pd.concat(pdf_strings)
pdf_strings_df['clean_string'] = pdf_strings_df['raw_string'].str.replace('\n', '').str.replace('\t', '').str.replace('\xa0', '')
pdf_strings_df = pdf_strings_df.drop(['raw_string'], axis = 1)
pdf_strings_df['clean_string'] = pdf_strings_df['clean_string'].str.strip()
pdf_strings_df.head()

10

,download_link,clean_string
0,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,File OF-Tolls-Group1-E101-2019-02 02 19 June 2...
0,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,"450 - 1st Street SW Calgary, Alberta T2P 5H1 ..."
0,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,File OF-Tolls-Group1-E101-2019-02 02 8 June 20...
0,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,File OF-Tolls-Group1-E101-2019-02 02 7 June 20...
0,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,File OF-Tolls-Group1-E101-2019-02 02 4 June 20...


In [29]:
%%time
# CPU times: user 21min 7s, sys: 45.2 s, total: 21min 52s
# Wall time: 21min 54s
sents_list = []

# for x, y in zip(df_all_record['clean string'][0:4], df_all_record['folder_link'][0:4]):
for x, y in zip(pdf_strings_df['clean_string'], pdf_strings_df['download_link']):
    try:
        doc = nlp(x)
        sentences = [x.text for x in doc.sents]
        df_ = pd.DataFrame(sentences, columns = ['sentence'])
        df_['download_link'] = str(y)
        sents_list.append(df_)
    except:
        print(y)

Wall time: 15.7 s


In [26]:
df_sents_new = pd.concat(sents_list)
df_sents_new.head()
df_sents_new.shape

,sentence,download_link
0,File OF-Tolls-Group1-E101-2019-02 02,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...
1,19 June 2021,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...
2,To: Canadian Shippers Group Distribution List,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...
3,MEG Energy Corp.,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...
4,Suncor Energy Inc.,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...


(3609, 2)

In [27]:
df_sents_new.to_csv('huhu.csv')

## Part 5: Topic specific NLP

In [13]:
%%time
pd.options.mode.chained_assignment = None  # default='warn'
# create dfs based on topic
# next steps: explore regex, + or - ten sentences in index to expand scope
# remove not sentences ex. classify is sentence is not sentence 

df_public_interest = df_sents_new[df_sents_new['sentence'].str.contains('public interest')]
df_public_interest['type'] = 'public interest'

df_competition = df_sents_new[df_sents_new['sentence'].str.contains('competitiveness|competitive|competition|compete')]#.reset_index(drop = True)
df_competition['type'] = 'competition'  # df.loc[:, 'newCol'] = df['Month']*2 still error hence chained_assignment = None

# df_reconciliation = df_sents_new[df_sents_new['sentence'].str.contains('Indigenous|First Nation|econciliation')]#.reset_index(drop = True)
# df_reconciliation['type'] = 'Reconciliation'

df_market_power = df_sents_new[df_sents_new['sentence'].str.contains('market power')]#.reset_index(drop = True)
df_market_power['type'] = 'market power'

df_tolling = df_sents_new[df_sents_new['sentence'].str.contains('tolling principle|just and reasonable|economic efficiency|market signal|discrimina|cross-subsidi')]#.reset_index(drop = True)
df_tolling['type'] = 'tolling'

df_risk = df_sents_new[df_sents_new['sentence'].str.contains('fundamental risk')]#.reset_index(drop = True)
df_risk['type'] = 'risk'

df_keystone = df_sents_new[df_sents_new['sentence'].str.contains('eystone|KXL')]#.reset_index(drop = True)
df_keystone['type'] = 'Keystone'

df_lakehead = df_sents_new[df_sents_new['sentence'].str.contains('Lakehead')]#.reset_index(drop = True)
df_lakehead['type'] = 'Lakehead'

df_trans_mountain = df_sents_new[df_sents_new['sentence'].str.contains('Trans Mountain|TMX')]#.reset_index(drop = True)
df_trans_mountain['type'] = 'Trans Mountain'

df_contract = df_sents_new[df_sents_new['sentence'].str.contains('contract')]#.reset_index(drop = True)
df_contract['type'] = 'contracting'

df_line9 = df_sents_new[df_sents_new['sentence'].str.contains('Line 9')]#.reset_index(drop = True)
df_line9['type'] = 'Line 9'

df_ROE = df_sents_new[df_sents_new['sentence'].str.contains('ROE')]#.reset_index(drop = True)
df_ROE['type'] = 'ROE'

df_innovative = df_sents_new[df_sents_new['sentence'].str.contains('innovative|innovation|innovate')]#.reset_index(drop = True)
df_innovative['type'] = 'innovation'

df_sentences_type = pd.concat([df_public_interest, df_competition, df_market_power, \
                               df_tolling, df_risk, df_keystone, df_lakehead, df_trans_mountain, df_contract, \
                               df_ROE, df_line9, df_innovative])
df_sentences_type.shape
df_sentences_type.to_csv('RH0012020_strings_sents_special.csv', index = False)
df_sentences_type.tail()

CPU times: user 3.18 s, sys: 13.5 ms, total: 3.19 s
Wall time: 3.19 s


,sentence,download_link,type
697,To understand the potential to stifle innovati...,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,innovation
1130,To understand the potential to stifle innovati...,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,innovation
332,"Other examples: Inthe capital goods industry, ...",https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,innovation
3113,"Would you agree that upstream producers, espec...",https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,innovation
1894,Where we moved away from strict cost of servic...,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,innovation


### Tokens, names entities, noun chunks

In [14]:
%%time 
# CPU times: user 4min 42s, sys: 1.15 s, total: 4min 43s
# Wall time: 4min 43s

tokens_list = []
ents_list = []
nouns_list = []
nlp.max_length = 2000000

for x , y, z in zip(df_sentences_type['sentence'], df_sentences_type['download_link'], df_sentences_type['type']): #testing all
# for x , y in zip(df['text_string'].to_list()[0:2], df['link'].to_list()[0:2]): #testing small
    doc = nlp(x)
    
    tokens = [(x.text, x.pos_, x.tag_, x.dep_, x.shape_, x.is_alpha, x.is_stop) for x in doc]
    df_tokens = pd.DataFrame(tokens, columns = ['token', 'pos', 'tag', 'dep', 'shape_', 'is_alpha', 'is_stop'])
    df_tokens['download_link'] = y
    df_tokens['sentence'] = x
    df_tokens['type'] = z
    tokens_list.append(hdf_tokens)
    
    ents = [(x.text, x.start_char, x.end_char, x.label_) for x in doc.ents]
    df_ents = pd.DataFrame(ents, columns = ['token', 'start_char', 'end_char', 'ent_label'])
    df_ents['download_link'] = y
    df_ents['sentence'] = x
    df_ents['type'] = z
    ents_list.append(df_ents)
    
    nouns = [x.text for x in doc.noun_chunks]
    df_nouns = pd.DataFrame(nouns, columns = ['noun_chunks'])
    df_nouns['download_link'] = y 
#     nouns = [(x.text, x.root.text, x.root.dep_, x.root.head.text) for x in doc.noun_chunks]
#     df_nouns = pd.DataFrame(nouns, columns = ['noun_chunks', 'root_text', 'root_dep', 'root_head'])
    df_nouns['sentence'] = x
    df_nouns['type'] = z
    nouns_list.append(df_nouns)

CPU times: user 4min 42s, sys: 1.15 s, total: 4min 43s
Wall time: 4min 43s


In [21]:
df_tokens_all = pd.concat(tokens_list)
df_tokens_all.head()
df_tokens_all = df_tokens_all[df_tokens_all['is_stop'] == False] #remove stop words
df_tokens_all = df_tokens_all[df_tokens_all['is_alpha'] == True] #remove 
df_tokens_all.shape
df_tokens_all.to_csv('RH0012020_tokens_sentspecial.csv', index = False)

df_ents_all = pd.concat(ents_list)
df_ents_all['explained'] = df_ents_all['ent_label'].apply(lambda x: spacy.explain(x))
df_ents_all.head()
df_ents_all.shape
df_ents_all.to_csv('RH0012020_ents_sentspecial.csv', index = False)

df_nouns_all = pd.concat(nouns_list)
df_nouns_all.head()
df_nouns_all.to_csv('RH0012020_nouns_sentspecial.csv', index = False)
df_nouns_all.shape

,noun_chunks,download_link,sentence,type
0,which crude oil price impacts,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,"d) Explain which crude oil price impacts, if a...",public interest
1,your business,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,"d) Explain which crude oil price impacts, if a...",public interest
2,the Canadian public interest,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,"d) Explain which crude oil price impacts, if a...",public interest
0,Attachment,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,Attachment to Commission Letter dated 1 Februa...,public interest
1,Commission Letter,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,Attachment to Commission Letter dated 1 Februa...,public interest


(274300, 4)

In [23]:
df_sents_new.head()

,sentence,download_link
0,File OF-Tolls-Group1-E101-2019-02 02 12 May 20...,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...
1,To: All Participants in the RH-001-2020 Procee...,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...
2,(Enbridge),https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...
3,Canadian Mainline Contracting Application Hear...,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...
4,"On 11 May 2021, the Commission of the Canada E...",https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...


In [24]:
%%time 
#Wall time: 18min 51s
tokens_list = []
ents_list = []
nouns_list = []
nlp.max_length = 2000000

for x , y in zip(df_sents_new['sentence'].to_list(), df_sents_new['download_link'].to_list()): #testing all
# for x , y in zip(df['text_string'].to_list()[0:2], df['link'].to_list()[0:2]): #testing small
    doc = nlp(x)
    
#     tokens = [(x.text, x.pos_, x.is_alpha, x.is_stop) for x in doc]
#     df_tokens = pd.DataFrame(tokens, columns = ['token', 'pos', 'is_alpha', 'is_stop'])
#     df_tokens['link'] = y
#     tokens_list.append(df_tokens)
    
    ents = [(x.text, x.label_) for x in doc.ents]
    df_ents = pd.DataFrame(ents, columns = ['token', 'ent_label'])
    df_ents['link'] = y
    df_ents['sentence'] = x
    ents_list.append(df_ents)
    
#     nouns = [x.text for x in doc.noun_chunks]
#     df_nouns = pd.DataFrame(nouns, columns = ['noun_chunks'])
#     df_ents['sentence'] = x
#     df_nouns['link'] = y 
#     nouns_list.append(df_nouns)

CPU times: user 23min 59s, sys: 4.52 s, total: 24min 3s
Wall time: 24min 6s


In [25]:
df_ents_all = pd.concat(ents_list)
df_ents_all['explained'] = df_ents_all['ent_label'].apply(lambda x: spacy.explain(x))
df_ents_all.head()
df_ents_all.shape
df_ents_all.to_csv('RH0012020_ents_all.csv', index = False)

,token,ent_label,link,sentence,explained
0,May 2021,DATE,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,File OF-Tolls-Group1-E101-2019-02 02 12 May 20...,Absolute or relative dates or periods
0,Enbridge Pipelines Inc.,ORG,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,To: All Participants in the RH-001-2020 Procee...,"Companies, agencies, institutions, etc."
0,Enbridge,ORG,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,(Enbridge),"Companies, agencies, institutions, etc."
0,Canadian Mainline Contracting Application Hear...,ORG,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,Canadian Mainline Contracting Application Hear...,"Companies, agencies, institutions, etc."
1,2020,DATE,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,Canadian Mainline Contracting Application Hear...,Absolute or relative dates or periods


(704046, 5)